Script structure:
* 0. Set parameters (will be part of pipeline so this block will be silenced)
  1. Setup:
        * Paths
        * Utils functions
        * Load and check config file
  2. Load Data
        * **Routine data** (DHIS2) already formatted & aggregated (output of pipeline XXX)
        * **Shapes** (DHIS2) for plotting (this could be removed if we move the plots to "report/EDA" nb)
  3. Calculate **Reportng Rate (RR)**
        * "Ousmane" way (old code) - find better name for this!
        * "WHO" / "Fre" way (based on code in nb: `~/dhis2_incidence/code/WIP/code_from_fre/DRC_DHIS2_analyses_fvdb_v2.ipynb`)
     **Export** reporting rate data to **Datasets** as .csv and .parquet files
  6. 🚧 (possibly) Expand reporting: **data inspection** (plots and summary tables) - this might go to **dedicated nb** ...

-------------------
**Naming harmonization to improve code readability**:

**Reporting Rate** data frames, based on different **methods**:
* follwo this structure: `reporting_rate_<method>_<periodicity>`. So:
    * **CONF** (Diallo 2025) : `reporting_rate_conf_month`
    * **ANY** (as "this code simply tests for _any_ indicator that is present"): `reporting_rate_any_month`

--------------------

## Parameters

👇 these are now ⚡**pipeline parameters**⚡!

In [1]:
# Parameters
SNT_ROOT_PATH = "/home/hexa/workspace"
REPORTING_RATE_THRESHOLD = 0.8


## 1. Setup

### 1.1. Paths

In [ ]:
# PROJECT PATHS
CODE_PATH <- file.path(SNT_ROOT_PATH, 'code') # this is where we store snt_functions.r and snt_utils.r
CONFIG_PATH <- file.path(SNT_ROOT_PATH, 'configuration') # .json config file
DATA_PATH <- file.path(SNT_ROOT_PATH, 'data') # same as in Datasets but /data/ gets over written every time a new version of Datasets is pushed

### 1.2. Utils functions

In [ ]:
source(file.path(CODE_PATH, "snt_utils.r"))

### 1.3. Packages

In [ ]:
# List required pcks  ---------------->  check  what are the really required libraries
required_packages <- c("arrow", # for .parquet
                       # "dplyr", "tidyr", "stringr", (just load tidyverse instead as I need more of the tidyverse packages)
                       "tidyverse",
                       "stringi", 
                       "sf",
                       "jsonlite", 
                       "httr", 
                       "reticulate")

# Execute function
install_and_load(required_packages)

### 1.3.1. OpenHEXA-specific settings

#### For 📦{sf}, tell OH where to find stuff ...

In [ ]:
# Hope this gets fixed at the source one day ...
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")

#### Set environment to load openhexa.sdk from the right path

In [ ]:
# Set environment to load openhexa.sdk from the right path
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

### 1.4. Load and check `config` file

In [ ]:
# Load SNT config

config_file_name <- "SNT_config.json" # ⚠️ The config file can be changed here if needed!
config_json <- tryCatch({
        jsonlite::fromJSON(file.path(CONFIG_PATH, config_file_name)) 
    },
    error = function(e) {
        msg <- paste0("Error while loading configuration", conditionMessage(e))  
        cat(msg)   
        stop(msg) 
    })

msg <- paste0("SNT configuration loaded from  : ", file.path(CONFIG_PATH, config_file_name))
log_msg(msg)

🚨 Config Validation in pipeline

In [ ]:
# # CHECK SNT configuration 
# snt_config_mandatory <- c("COUNTRY_CODE", "DHIS2_ADMINISTRATION_1", "DHIS2_ADMINISTRATION_2") #, "ORG_UNITS_LEVELS_SELECTION")
# for (conf in snt_config_mandatory) {
#     print(paste(conf, ":", config_json$SNT_CONFIG[conf]))
#     if (is.null(config_json$SNT_CONFIG[[conf]])) {
#         msg <- paste("Missing configuration input:", conf)
#         cat(msg)   
#         stop(msg)
#     }
# }

**Save config fields as variables**

In [ ]:
# Generic
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
ADMIN_1 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_1)
ADMIN_2 <- toupper(config_json$SNT_CONFIG$DHIS2_ADMINISTRATION_2)

# Specific to INCIDENCE calculation (this nb)
# How to treat 0 values (in this case: "SET_0_TO_NA" converts 0 to NAs)
NA_TREATMENT <- config_json$SNT_CONFIG$NA_TREATMENT

# Which (aggregated) indicators to use to evaluate "activity" of an HF - for Reporting Rate method "Ousmane"
DHIS2_INDICATORS <- names(config_json$DHIS2_DATA_DEFINITIONS$DHIS2_INDICATOR_DEFINITIONS)

In [ ]:
# Fixed routine formatting columns
fixed_cols <- c('OU','PERIOD', 'YEAR', 'MONTH', 'ADM1_ID', 'ADM1', 'ADM2_ID', 'ADM2') # use `OU` as it contains unique ids (OU_NAME has homonimous values!)
print(paste("Fixed routine data (\"dhis2_routine\") columns (always expected): ", paste(fixed_cols, collapse=", ")))

## 2. Load Data

### 2.1. **Routine** data (DHIS2) 
already formatted & aggregated (output of pipeline XXX)

In [ ]:
# DHIS2 Dataset extract identifier
dataset_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

# Load file from dataset
dhis2_routine <- tryCatch({ get_latest_dataset_file_in_memory(dataset_name, paste0(COUNTRY_CODE, "_routine.parquet")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 routine data file for: " , COUNTRY_CODE, conditionMessage(e))  # log error message
                      cat(msg)
                      stop(msg)
})

msg <- paste0("DHIS2 routine data loaded from dataset : ", dataset_name, " dataframe dimensions: ", paste(dim(dhis2_routine), collapse=", "))
log_msg(msg)

In [ ]:
head(dhis2_routine)

### 2.2. Shapes for plotting maps (choropleths)

In [ ]:
# DHIS2 Dataset extract identifier
dataset_name <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

# Load file from dataset
dhis2_shapes <- tryCatch({ get_latest_dataset_file_in_memory(dataset_name, paste0(COUNTRY_CODE, "_shapes.geojson")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 shapes data file for: " , COUNTRY_CODE, conditionMessage(e))  # log error message
                      cat(msg)
                      stop(msg)
})

msg <- paste0("DHIS2 shapes data loaded from dataset : ", dataset_name, " dataframe dimensions: ", paste(dim(dhis2_shapes), collapse=", "))
log_msg(msg)

In [ ]:
# `head()` cannot display, needs ‘geojsonio’ (which I cannot install) so let's just check col names ... 
names(dhis2_shapes)

## 3. Calculate **Reporting Rate** (RR)
We compute it using 2 approaches, user can decided later on which one to use for incidence adjustment.

### 3.1. Method "**ANY**: tests for **_any_ indicator** (as defined in **config file**) that is present
**_Ousmane's (cleaned) old code from BFA SNT process_**

🚨 **Note**: **updated** approach to define "Active" health facilities - **how to define list of indicators to consider:** 🚨
* **Old (Ousmane's code)**: used indicators (columns in routine data) defined in the code as `report_cols = c("SUSP", "TEST", "CONF", "PRES", "PRESSEV", "MALTREAT", "MALADM", "MALDTH")`
    * Problem: _what to do if any of these are missing?_ Example: in current BFA data, "PRESSEV" is missing, so code breaks 
* **Current** - applied here: instead of `report_cols`, use the list of indicators defined in the config file, as: `DHIS2_INDICATORS <- names(config.json$DHIS2_DATA_DEFINITIONS$DHIS2_INDICATOR_DEFINITIONS)` (`DHIS2_INDICATORS` is defined at begining of nb)

#### Define cols used to evaluate HF "activity" (whether a HF is reporting or not)

In [ ]:
cols_to_subset <- c(fixed_cols, DHIS2_INDICATORS)
print(cols_to_subset)

dhis2_routine_subset = dhis2_routine %>% 
  dplyr::select(all_of(cols_to_subset))  # old: select(all_of(c(fixed_cols, DHIS2_INDICATORS)))

#### 🚨 Set `0` values to `NA`

In [ ]:
# 0 value to NA 
if (NA_TREATMENT == 'SET_0_TO_NA') { 
    dhis2_routine_subset[, DHIS2_INDICATORS][dhis2_routine_subset[, DHIS2_INDICATORS] == 0] <- NA  
    print("Set 0 values to NA")
}

In [ ]:
# HF considered "inactif" when all indicators are NA (= did not submit anything for these indicators), 
#     else "actif" (= they submitted something)
hf_active = dhis2_routine_subset %>%
    dplyr::mutate(nomiss = apply(dhis2_routine_subset[,DHIS2_INDICATORS], 1, function(y) sum(!is.na(y))), 
                  varmis =ifelse(nomiss == 0, 0, 1),
                  ACTIVE = ifelse(varmis == 0, 'inactive', 'active')) %>% # colname was "active"
    dplyr::arrange(ADM1, ADM2, OU, PERIOD) %>% # OU,
    dplyr::group_by(ADM1, ADM2, OU) %>% # OU
    dplyr::mutate(cummiss = sum(nomiss), 
                  inactivity = nomiss/length(DHIS2_INDICATORS)*100, #------------------------> this used to be hardcoded as `nomiss/3`
                  start_date = ifelse(
                    any(inactivity != 100, na.rm = TRUE),
                    min(PERIOD[inactivity != 100], na.rm = TRUE),
                    NA  # Default to NA if no valid values
                    )) %>%
    dplyr::filter(PERIOD >= start_date)

In [ ]:
head(hf_active, 3)

In [ ]:
# hf_active_rate = hf_active %>%   # old
reporting_rate_any_month = hf_active %>% 
    dplyr::group_by(ADM2, YEAR, MONTH) %>% # GP replaced PERIOD with MONTH for clarity and consistency with other `reporting_rate_xxx_month`
    dplyr::summarize(TOTAL_HF = length(OU),
                     TOTAL_HF_ACTIVE = length(which(ACTIVE == 'active')), 
                     .groups = "drop") %>%
    #💡 keep `REPORTING _RATE` as (0-1) as later is divided by 100 to make `rep_rate` anyways ... 
    dplyr::mutate(REPORTING_RATE = round(TOTAL_HF_ACTIVE/TOTAL_HF,2), # was `round(TOTAL_HF_ACTIVE/TOTAL_HF*100,2)`
                  REPORTING_RATE_QUALITY = ifelse(REPORTING_RATE >= REPORTING_RATE_THRESHOLD, 'good', 'bad')) %>%
    ungroup() %>%  # 🚨 GP added 20250522!
    mutate(YEAR = as.integer(YEAR),
           MONTH = as.integer(MONTH),
          ) # 🚨 GP added 20250522!

In [ ]:
head(reporting_rate_any_month, 3)

#### Plot by MONTH (heatmap)

In [ ]:
# Plot heatmap
options(repr.plot.width = 20, repr.plot.height = 10)

reporting_rate_any_month %>%
mutate(
    DATE = as.Date(paste(YEAR, MONTH, "01", sep = "-")), # GP note: as.Date() works when YEAR and MONTH are integers as well as character ... (no need to convert)
    ADM2 = factor(ADM2)
    # reporting_rate = REPORTING_RATE / 100  # convert % to fraction if needed  # GP: a bit silly to do and undo in the next step ...
    ) %>%
ggplot(., 
       aes(x = DATE, y = ADM2, 
           fill = REPORTING_RATE * 100) 
      ) + 
  geom_tile() +
  scale_fill_viridis_c(
    option = "C",
    direction = 1,
    limits = c(0, 100), 
    name = "Reporting rate (%)"
  ) +
  labs(
    title = "Taux de rapportage mensuel par district sanitaire",
    subtitle = "Chaque tuile représente l’exhaustivité du rapportage par district et par mois",
    x = "Mois",
    y = "District sanitaire"
  ) +
  theme_minimal(base_size = 13) +
  theme(
    axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5, size = 9),
    axis.text.y = element_text(size = 9),
    plot.title = element_text(face = "bold", hjust = 0.5, size = 14),
    plot.subtitle = element_text(hjust = 0.5, size = 12),
    legend.position = "right",
    panel.grid = element_blank()
  )

#### Reporting Rate: **year**ly **median** per **ADM2**
GP: Fre, it's actually **mean** not median ... ! ⚠️

In [ ]:
reporting_rate_any_year = reporting_rate_any_month %>%
    group_by(ADM2, YEAR) %>% 
    summarise(REPORTING_RATE = round(mean(REPORTING_RATE, na.rm = T), 2), .groups = "drop") %>% # GP: shouldn't it be `median()`?
    ungroup() %>%
    mutate(YEAR = as.integer(YEAR)) # 🚨 GP added 20250522!

In [ ]:
print(dim(reporting_rate_any_year))
head(reporting_rate_any_year, 3)

----------------------------

### 3.2. Method **CONF**: based on reporting of **confirmed cases**
**_Reporting rate following methods by WHO and as per Diallo_2025 paper_**

To accurately measure data completeness, we calculate the monthly reporting rate per health district (ADM2) as the **proportion of facility–months that submitted at least one report containing a confirmed malaria case** (**CONF**). <br>
For each ADM2, we expect one report per facility per month. For example, if an ADM2 has 25 facilities, we expect 25 reports for a given month. If only 21 of those facilities report confirmed cases that month, the reporting rate is 21/25 = 84%.

This method improves over simple binary completeness flags by accounting for both spatial (facility coverage) and temporal (monthly timeliness) dimensions. A facility-month is **considered reported** if the **CONF value is not missing**, which serves as a proxy for overall completeness of malaria indicators. We use the presence of CONF (confirmed malaria cases) as the condition for marking a facility-month as reported because it is a core indicator consistently tracked across the dataset. This choice ensures alignment with the structure of the incidence calculation, which is also mainly based on confirmed cases.

In [ ]:
head(dhis2_routine, 3)

#### Calculate

In [ ]:
# Tag as "REPORTED" only if `CONF` is not NA
# rds_reporting <- rds %>% # Fre's
dhis2_routine_reporting <- dhis2_routine %>%
  mutate(REPORTED_CONF = if_else(!is.na(CONF), 1, 0))

In [ ]:
# Calculate at ADM2 × MONTH level
# reporting_rate_conf_monthly <- rds_reporting %>% # Fre's
reporting_rate_conf_month <- dhis2_routine_reporting %>%
  group_by(ADM2, YEAR, MONTH) %>% 
  summarise(
    N_FACILITIES = n_distinct(OU),
    N_REPORTS = sum(REPORTED_CONF, na.rm = TRUE),
    REPORTING_RATE = N_REPORTS / N_FACILITIES,
    .groups = "drop"
  ) %>%
  ungroup() %>%  # 🚨 GP added 20250522!
    mutate(YEAR = as.integer(YEAR),
           MONTH = as.integer(MONTH)
          ) # 🚨 GP added 20250522!

head(reporting_rate_conf_month, 3)

In [ ]:
# Aggregate monthly to year
reporting_rate_conf_year <- reporting_rate_conf_month %>%
  group_by(ADM2, YEAR) %>%
  summarise(
      N_FACILITIES = sum(N_FACILITIES, na.rm = TRUE),
      N_REPORTS = sum(N_REPORTS, na.rm = TRUE),
      REPORTING_RATE = N_REPORTS / N_FACILITIES,
    .groups = "drop"
  ) %>% 
   ungroup() %>%  # GP added 20250522!
    mutate(YEAR = as.integer(YEAR)) # GP added 20250522!

head(reporting_rate_conf_year) 

#### Plot by MONTH (heatmap)

In [ ]:
# Plot reporting rate heatmap
options(repr.plot.width = 20, repr.plot.height = 10) 

reporting_rate_conf_month %>%
mutate(
    DATE = as.Date(paste0(YEAR, "-", MONTH, "-01"))
    ) %>%
ggplot(., aes(x = DATE,  # GP replaced `date` with `DATE`
              y = factor(ADM2), # GP replaced `y = ADM2` with `y = factor(ADM2)`
              fill = REPORTING_RATE * 100)
      ) + 
  geom_tile() +
  scale_fill_viridis_c(
    option = "C",
    direction = 1,  # blue = low, yellow = high
    limits = c(0, 100),
    name = "Reporting rate (%)"
  ) +
  labs(
    title = "Monthly Reporting Rate by Health District",
    subtitle = "Each tile represents the reporting completeness per district per month",
    x = "Month",
    y = "Health District"
  ) +
  theme_minimal(base_size = 13) +
  theme(
    axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5, size = 9),
    axis.text.y = element_text(size = 9),
    plot.title = element_text(face = "bold", hjust = 0.5, size = 14),
    plot.subtitle = element_text(hjust = 0.5, size = 12),
    legend.position = "right",
    panel.grid = element_blank()
  )

#### Plot by YEAR (choropleth)

In [ ]:
# 1. Summarize SNIS reporting rates by ADM2 and year (if not already done)
# reporting_rate_conf_yearly <- reporting_rate_conf_monthly %>%
reporting_rate_conf_yearly <- reporting_rate_conf_month %>%
  group_by(ADM2, YEAR) %>%
  summarise(REPORTING_RATE = mean(REPORTING_RATE, na.rm = TRUE), .groups = "drop")

head(reporting_rate_conf_yearly, 3)

--------------------------

# ⚠️ TO DO: modify code to have `reporting_rate_conf_year` and `reporting_rate_any_year` calculated in exactly the same way! ⚠️
At the moment: one uses `sum()` and the other `mean()`. Decide which approach to keep for consistency ... !

Old: 🤔 Q fro Fre
Here you are re-doing the **same summarization (month -> year)** as above but slightly different (`sum()` n_reports and n_facilities vs `mean()` of reporting_rate).

Just FYI, numbers a slightly off on the 3rd decimal (prob due to rounding). 

Here's a comparison of the 2 tables (note that I started renaming df's)

In [ ]:
# same same but different (not really)
head(select(reporting_rate_conf_year, ADM2, YEAR, REPORTING_RATE), 3) # renamed to be consistent with rest of script

head(reporting_rate_conf_yearly, 3) # kept original name of df

--------------------------

In [ ]:
# 2. Join ADM2 shapes with SNIS reporting data
map_data <- dhis2_shapes %>% 
  left_join(reporting_rate_conf_year, by = "ADM2") %>% # GP: was "reporting_rate_conf_yearly"
  sf::st_as_sf() 

In [ ]:
# 3. Bin reporting rate values
map_data <- map_data %>%
  mutate(rate_cat = case_when(
    REPORTING_RATE < 0.5 ~ "< 50%",
    REPORTING_RATE < 0.8 ~ "50–80%",
    REPORTING_RATE < 0.9 ~ "80–90%",
    REPORTING_RATE >= 0.9 ~ "90–100%"  # includes 100%
  ))

# 4. Define colors
rate_colors <- c(
  "< 50%"     = "#b2182b",  # dark red
  "50–80%"    = "#f46d43",  # reddish-orange, more vibrant
  "80–90%"    = "#fee08b",  # yellow
  "90–100%" = "#4daf4a"  # clear, strong green (used in many R palettes)
)

# 5. Plot
options(repr.plot.width = 20, repr.plot.height = 5)
ggplot(map_data) +
  geom_sf(aes(fill = rate_cat,
             geometry = geometry),
          color = "white", size = 0.2) +
  facet_wrap(~ YEAR, nrow = 1) +
  scale_fill_manual(values = rate_colors, name = "Taux de rapportage") +
  labs(title = "Taux de rapportage par district sanitaire (ADM2), par année") +
  theme_minimal(base_size = 14) +
  theme(
    strip.text = element_text(size = 16),
    plot.title = element_text(size = 18, hjust = 0.5),
    legend.position = "bottom",
    panel.spacing = unit(0.2, "lines"),
    axis.text = element_blank(),
    axis.title = element_blank(),
    axis.ticks = element_blank()
  ) +
  coord_sf(datum = NA)

# 4. Export

## 4.1. 📁 To /data/ folder

#### CSV

In [ ]:
# Method "ANY"
write_csv(reporting_rate_any_month,
          file.path(DATA_PATH, "dhis2_reporting_rate", paste0(COUNTRY_CODE, "_reporting_rate_any_month.csv"))      
         )

# Method "CONF"
write_csv(reporting_rate_conf_month,
          file.path(DATA_PATH, "dhis2_reporting_rate", paste0(COUNTRY_CODE, "_reporting_rate_conf_month.csv"))      
         )

#### parquet

In [ ]:
# Method "ANY"
arrow::write_parquet(reporting_rate_any_month,
                     file.path(DATA_PATH, "dhis2_reporting_rate", paste0(COUNTRY_CODE, "_reporting_rate_any_month.parquet"))
                    )

# Method "CONF"
arrow::write_parquet(reporting_rate_conf_month,
                     file.path(DATA_PATH, "dhis2_reporting_rate", paste0(COUNTRY_CODE, "_reporting_rate_conf_month.parquet"))
                    )
